In [27]:
import pandas as pd
from py2neo import Graph, GraphService
from neo4j import GraphDatabase
import sys, os


In [28]:
module_dir = os.getcwd().replace("notebook", "src")
if module_dir not in sys.path:
    sys.path.append(module_dir)
from common import utils
from common.database import *

In [29]:
# graph_staging = Graph('bolt://34.67.212.125:7687', auth=('neo4j', 'Lifelike0.9staging'))
# graph_dtu = Graph('bolt+s://kg.biosustain.dtu.dk:7687', auth=('robin', 'kTxu$drJ%3C3^cHk'))
graph = Graph('bolt://localhost:7687', auth=('neo4j', 'rcai'))

# Create new protein LMDB file (LL-3056)

#### add uniprot protein name and id as synonym

```
call apoc.periodic.iterate(
    "match(n:db_UniProt) return n",
    "merge (s:Synonym {name:n.name}) merge (n)-[:HAS_SYNONYM]->(s)",
    {batchSize: 10000}
);

call apoc.periodic.iterate(
    "match(n:db_UniProt) return n",
    "merge (s:Synonym {name:n.id}) merge (n)-[:HAS_SYNONYM]->(s)",
    {batchSize: 10000}
);

call apoc.periodic.iterate(
"match(n:db_UniProt)-[:HAS_TAXONOMY]-(t) return n, t",
"set n.tax_id = t.id",
{batchSize: 5000}
)
```

In [30]:
query = """
match(n:db_UniProt)-[:HAS_SYNONYM]-(s) 
return n.id as id, n.name as name, s.name as synonym, n.tax_id as tax_id, n.data_source as data_source
"""
df = graph.run(query).to_data_frame()
print(len(df))
df.to_csv('/Users/rcai/data/notebook/uniprot/ProteinForLMDB_20210527.tsv', sep='\t', index=False)

2643366


In [26]:
#df.to_csv('/Users/rcai/data/notebook/uniprot/ProteinForLMDB_04062021.tsv.gz', sep='\t', index=False, compression='gzip')

## Add BioCyc(Ecoli and Human) protein synonyms as Uniprot Protein Synonyms (LL-3080)

In [11]:
# check biocyc synonyms that are not uniprot synonyms
query = """
match(n:Protein:db_BioCyc)-[:ENCODES]-()-[:IS]-()-[:HAS_GENE]-(p:db_UniProt) 
where 'db_EcoCyc' in labels(n) or 'db_HumanCyc' in labels(n)
with n, p match (p)-[:HAS_SYNONYM]-(s) with n, p, collect(s) as psyn 
match (n)-[:HAS_SYNONYM]-(s) where not s in psyn 
return p.id as uniprotId, p.name as uniprotName, n.id as biocycId, n.name as biocycName, s.name as biocycSyn 
"""
graph = Graph('bolt://localhost:7687', auth=('neo4j', 'rcai'))
df = graph.run(query).to_data_frame()
print(len(df))
df.to_csv('/Users/rcai/data/notebook/uniprot/biocyc_syns_notInUniprot.tsv', sep='\t', index=False)
graph.close()

22973


Many biocyc protein synonym is EC number, e.g. 3.4.24.-. Remove those from Synonym nodes. Also some NCBI genes have a synonym contains no non-digit character. 

Query:
```
match(n:Synonym) where n.name =~ '[^a-zA-Z]*' detach delete n
```

In [20]:
# get only single word synonym
query = """
match(n:Protein:db_BioCyc)-[:ENCODES]-()-[:IS]-(g)-[:HAS_GENE]-(p:db_UniProt) 
where 'db_EcoCyc' in labels(n) or 'db_HumanCyc' in labels(n)
with n, g, p match (p)-[:HAS_SYNONYM]-(s) with n, g, p, collect(s) as psyn 
match (n)-[:HAS_SYNONYM]-(s) where not s in psyn and s.name =~ '[\w-]*'
return p.id as uniprotId, p.name as uniprotName, p.gene_name as uniprotGeneName, g.id as geneId, g.name as geneName, s.name as biocycSyn
"""
graph = Graph('bolt://localhost:7687', auth=('neo4j', 'rcai'))
df = graph.run(query).to_data_frame()
print(len(df))
df.to_excel('/Users/rcai/data/notebook/uniprot/biocyc_simple_syns_ForUniprot.xlsx', index=False)


7118


##### get only single word synonym, and filter out synonyms linked to E coli insert genes (e.g. insA1, insA2, insA3) due to many-many relationships


In [26]:
query = """
match(n:db_EcoCyc:Gene) where n.name =~'ins[A-Z][0-9]+' with collect(n) as filteredGenes 
match(n:Protein:db_BioCyc)-[:ENCODES]-(g)-[:IS]-()-[:HAS_GENE]-(p:db_UniProt) 
where ('db_EcoCyc' in labels(n) or 'db_HumanCyc' in labels(n)) and not g in filteredGenes
with n, g, p match (p)-[:HAS_SYNONYM]-(s) with n, g, p, collect(s) as psyn 
match (n)-[:HAS_SYNONYM]-(s) where not s in psyn and s.name =~ '[\w-]*'
return p.id as uniprotId, p.name as uniprotName, p.gene_name as uniprotGeneName, g.name as matchedGeneName, s.name as biocycSyn
"""
graph = Graph('bolt://localhost:7687', auth=('neo4j', 'rcai'))
df = graph.run(query).to_data_frame()
print(len(df))
df.to_excel('/Users/rcai/data/notebook/uniprot/biocyc_simple_syns_ForUniprot_filtered.xlsx', index=False)


6809


#### Add biocyc protein synonyms as uniprot synonyms
```
match(n:db_EcoCyc:Gene) where n.name =~'ins[A-Z][0-9]+' with collect(n) as filteredGenes 
match(n:Protein:db_BioCyc)-[:ENCODES]-(g)-[:IS]-()-[:HAS_GENE]-(p:db_UniProt) 
where ('db_EcoCyc' in labels(n) or 'db_HumanCyc' in labels(n)) and not g in filteredGenes
with n, g, p match (p)-[:HAS_SYNONYM]-(s) with n, g, p, collect(s) as psyn 
match (n)-[:HAS_SYNONYM]-(s) where not s in psyn and s.name =~ '[\w-]*'
merge (p)-[:HAS_SYNONYM]->(s)
```